In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, classification_report, confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Define ConStant

In [ ]:
TRAIN_CSV = "train.csv"
VAL_CSV   = "val.csv"
TRAIN_DIR = "train"
VAL_DIR   = "val"

NUM_CLASSES = 5
BATCH_SIZE  = 32
EPOCHS      = 40
SEED        = 42

IMAGE_SIZE  = 224          
WEIGHT_DECAY = 1e-4
LR          = 1e-3

AUTOTUNE = tf.data.AUTOTUNE
tf.random.set_seed(SEED)

## Prepare Data

In [3]:
train_df = pd.read_csv(TRAIN_CSV)
val_df   = pd.read_csv(VAL_CSV)

In [4]:
train_df["filepath"] = train_df["file_name"].apply(lambda x: os.path.join(TRAIN_DIR, x))
val_df["filepath"]   = val_df["file_name"].apply(lambda x: os.path.join(VAL_DIR, x))

In [5]:
def decode_and_resize(path):
    img_bytes = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img_bytes, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)  # [0,1]
    img = tf.image.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    return img

In [ ]:
def augment(img):
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    img = tf.image.rot90(img, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    img = tf.image.random_brightness(img, max_delta=0.2)
    return img


In [7]:
def preprocess_train(path, label):
    img = decode_and_resize(path)
    img = augment(img)
    return img, tf.cast(label, tf.int32)

In [8]:
def preprocess_val(path, label):
    img = decode_and_resize(path)
    return img, tf.cast(label, tf.int32)

In [9]:
def make_dataset(df, training=True):
    paths  = df["filepath"].values
    labels = df["label"].values
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if training:
        ds = ds.shuffle(buffer_size=len(df), seed=SEED, reshuffle_each_iteration=True)
        ds = ds.map(preprocess_train, num_parallel_calls=AUTOTUNE)
    else:
        ds = ds.map(preprocess_val, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return ds



In [10]:
train_ds = make_dataset(train_df, training=True)
val_ds   = make_dataset(val_df, training=False)

## Build Model

In [ ]:
def build_transfer_model():
    #Input 224x224x3
    inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    
    x = layers.Rescaling(255.0)(inputs)
    
    # load model EfficientNetV2B0
    base_model = tf.keras.applications.EfficientNetV2B0(
        include_top=False,      
        weights="imagenet",     
        input_tensor=x,
        include_preprocessing=True
    )
    
    # Freeze
    base_model.trainable = False 
    
    # create 5 class
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)
    
    model = keras.Model(inputs, outputs, name="efficientnet_transfer")
    return model

In [20]:
model = build_transfer_model()

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 11s 0us/step


In [21]:
model.summary()

Model: "efficientnet_transfer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ rescaling[0][0]   │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          0 │ rescaling_1[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ normalization[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │      4,608 │ stem_activation[… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 112, 112,  │         64 │ block1a_project_… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_ac… │ (None, 112, 112,  │          0 │ block1a_project_… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_conv │ (None, 56, 56,    │      9,216 │ block1a_project_… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_bn   │ (None, 56, 56,    │        256 │ block2a_expand_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_act… │ (None, 56, 56,    │          0 │ block2a_expand_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_project_co… │ (None, 56, 56,    │      2,048 │ block2a_expand_a… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_project_bn  │ (None, 56, 56,    │        128 │ block2a_project_… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_expand_conv │ (None, 56, 56,    │     36,864 │ block2a_project_… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_expand_bn   │ (None, 56, 56,    │        512 │ block2b_expand_c

 Total params: 5,925,717 (22.60 MB)

 Trainable params: 6,405 (25.02 KB)

 Non-trainable params: 5,919,312 (22.58 MB)

In [22]:
optimizer = keras.optimizers.AdamW(learning_rate=LR, weight_decay=WEIGHT_DECAY)
loss_fn = keras.losses.SparseCategoricalCrossentropy()

In [23]:
model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=["accuracy"],
)

In [24]:
callbacks = [
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=3, verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=7, restore_best_weights=True, verbose=1
    ),
]

history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=callbacks,
)

Epoch 1/40
986/986 ━━━━━━━━━━━━━━━━━━━━ 692s 694ms/step - accuracy: 0.6911 - loss: 0.8355 - val_accuracy: 0.5280 - val_loss: 1.1676 - learning_rate: 0.0010
Epoch 2/40
986/986 ━━━━━━━━━━━━━━━━━━━━ 550s 558ms/step - accuracy: 0.7481 - loss: 0.6792 - val_accuracy: 0.5319 - val_loss: 1.1661 - learning_rate: 0.0010
Epoch 3/40
986/986 ━━━━━━━━━━━━━━━━━━━━ 566s 574ms/step - accuracy: 0.7575 - loss: 0.6485 - val_accuracy: 0.5245 - val_loss: 1.2124 - learning_rate: 0.0010
Epoch 4/40
986/986 ━━━━━━━━━━━━━━━━━━━━ 638s 647ms/step - accuracy: 0.7646 - loss: 0.6323 - val_accuracy: 0.5564 - val_loss: 1.1115 - learning_rate: 0.0010
Epoch 5/40
986/986 ━━━━━━━━━━━━━━━━━━━━ 592s 601ms/step - accuracy: 0.7666 - loss: 0.6244 - val_accuracy: 0.5657 - val_loss: 1.1138 - learning_rate: 0.0010
Epoch 6/40
986/986 ━━━━━━━━━━━━━━━━━━━━ 562s 570ms/step - accuracy: 0.7673 - loss: 0.6228 - val_accuracy: 0.5510 - val_loss: 1.1478 - learning_rate: 0.0010
Epoch 7/40
986/986 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step - accuracy

## Evaluation

In [30]:
y_true_val = []
y_pred_val = []

for batch_x, batch_y in val_ds:
    probs = model.predict(batch_x, verbose=0)
    preds = np.argmax(probs, axis=1)
    y_pred_val.extend(preds)
    y_true_val.extend(batch_y.numpy())

y_true_val = np.array(y_true_val)
y_pred_val = np.array(y_pred_val)

macro_f1_val = f1_score(y_true_val, y_pred_val, average="macro")
print("\n=== Validation macro-F1 ===")
print(macro_f1_val)

print("\n=== Validation classification report ===")
print(classification_report(
    y_true_val,
    y_pred_val,
    target_names=[str(i) for i in range(NUM_CLASSES)],
    digits=4
))

print("Confusion matrix:")
print(confusion_matrix(y_true_val, y_pred_val))


=== Validation macro-F1 ===
0.5477976226850141

=== Validation classification report ===
              precision    recall  f1-score   support

           0     0.5172    0.9299    0.6647      3436
           1     0.6238    0.6812    0.6512      4194
           2     0.6221    0.4396    0.5152      6681
           3     0.2310    0.3994    0.2927      2912
           4     0.8530    0.4811    0.6152      7549

    accuracy                         0.5564     24772
   macro avg     0.5694    0.5862    0.5478     24772
weighted avg     0.6322    0.5564    0.5633     24772

Confusion matrix:
[[3195   83   91   56   11]
 [ 518 2857  390  275  154]
 [ 781  280 2937 2423  260]
 [ 563  204  781 1163  201]
 [1121 1156  522 1118 3632]]


In [ ]:
print("unique preds:", np.unique(y_pred_val, return_counts=True))
print("unique true:", np.unique(y_true_val, return_counts=True))

unique preds: (array([0, 1, 2, 3, 4]), array([5939, 3963, 7258, 3506, 4106]))
unique true: (array([0, 1, 2, 3, 4], dtype=int32), array([3436, 4194, 6681, 2912, 7549]))


## Save model

In [ ]:
MODEL_PATH = "Task4.keras"
print(f"\nSaving model to {MODEL_PATH} ...")
model.save(MODEL_PATH)
print("Done.")

## Test

## Import Data

In [26]:
TEST_CSV   = "test_refined.csv"
TEST_DIR = "test"

In [27]:
test_df = pd.read_csv(TEST_CSV,index_col='id')
test_df["filepath"] = test_df["file_name"].apply(lambda x: os.path.join(TEST_DIR, x))

## Prepare Data

In [ ]:
def make_test_dataset(df):
    paths = df["filepath"].values
    ds = tf.data.Dataset.from_tensor_slices(paths)
    
    def preprocess_test(path):
        return decode_and_resize(path)

    ds = ds.map(preprocess_test, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return ds

test_ds_clean = make_test_dataset(test_df)

## Predict

In [ ]:
print("\nPredicting on test set")
all_pred_idx = []

for batch_images in test_ds_clean:
    probs = model.predict(batch_images, verbose=0)
    preds = np.argmax(probs, axis=1)
    all_pred_idx.extend(preds)

all_pred_idx = np.array(all_pred_idx, dtype=int)

## Save output

In [ ]:
pred_df = pd.DataFrame({
    "Task4": all_pred_idx
})

print("Prediction completed.")
pred_df.head()

In [ ]:
pred_df.to_csv("task4.csv", index=False)